<a href="https://colab.research.google.com/github/kally1218/pycharmproject/blob/main/%E8%87%AA%E5%8A%A8derivative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
#函数关于列向量x求导, 创建一个向量y 有浮点数的那种 0 ,1, 2 ,3
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
#找一个地方存y的梯度(导数),这是 PyTorch 中一个非常重要的步骤，
#当你想要计算某个张量（例如 x）相对于另一个张量的梯度（导数）时，就需要这样做。
#requires_grad_ 方法当传入 True 时，会原地修改 x 这个张量。
#这告诉 PyTorch 需要追踪所有对 x 执行的操作。这种追踪对于后续在反向传播过程中计算梯度是必需的
x.requires_grad_(True) #等价于 x= torch.arange(4,0, requires_grad=True)
x.grad #默认值是None, 以后这个能访问梯度,梯度就存在这个x.grad里面

In [3]:
#计算y, 显示y是反向传播
y = 2 * torch.dot(x, x) #torch,dot就是向量的內积写法,內积是个scalar哦!!!
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
y.backward() #计算y关于x每个分量的梯度
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
#验证梯度
x.grad == 4 * x

tensor([True, True, True, True])

In [6]:
#before calculate next gradient, we need to clean previous one
x.grad.zero_() #进行一个清除,记住有个下划线!!用于重写内容!!!!
#求x的总和
y = x.sum()
y.backward()
x.grad


tensor([1., 1., 1., 1.])

In [7]:
#向量的乘法
#深度学习中, 调用向量的反向计算的时候,一般, 会计算一个batch training sample中的每个组成部分的损失函数的导数
x.grad.zero_()
y = x * x
#等价于y.backward(torch.ones(len(x)))
y.sum().backward() #一般情况先对y求和,再反向求导
x.grad

tensor([0., 2., 4., 6.])

In [9]:
x.grad.zero_()
y = x * x
u = y.detach() #把y这个向量变成标量,之后要固定网络中的参数时候用, u变成常数了
z = u * x #这里u就变成常数了

z.sum().backward()
x.grad == u


tensor([True, True, True, True])

In [10]:
#用python的控制流,还是能求导
#PyTorch 会在 运行时 构建计算图（不是提前构建好）
#你写什么操作，它就记录什么操作（即“边写边构建”），操作完成后，
#计算图就完整了，可以用 .backward() 自动求导。
def f(a):
  b = a * 2
  while b.norm() < 1000: #norm默认范数2, 开平方取根号的公式
    b = b * 2
    if b.sum() > 0:
      c = b
    else:
      c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True) #a是input的随机数,size里面啥也不写等于标量的)
d = f(a)
d.backward()

a.grad == d / a #右边的式子是因为d的函数是a * 2 *....无限乘,所以,d的导数是个关于k个2的标量


tensor(True)